In [1]:
import tensorflow as tf
from tensorflow.python.lib.io import file_io
from tqdm.notebook import tqdm
import numpy as np

2023-11-17 22:00:08.520535: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 22:00:11.200137: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /global/software/sl-7.x86_64/modules/langs/cuda/11.2/lib64:/global/home/groups/consultsw/sl-7.x86_64/modules/cudnn/7.0.5-cuda9.0/lib64:/global/software/sl-7.x86_64/modules/langs/python/3.7/lib
2023-11-17 22:00:11.200290: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open s

In [2]:
tfr_filenames = [f'train-0-{num}.tfr' for num in range(133)]+[f'valid-0-{num}.tfr' for num in range(9)]+[f'test-0-{num}.tfr' for num in range(8)]

In [3]:
class Parser:
    def __init__(self, seq_length=524288, target_length=896, num_targets=5313, seq_1hot=True, seq_depth=4, seq_length_crop=None):
        self.seq_length = seq_length
        self.target_length = target_length
        self.num_targets = num_targets
        self.seq_1hot = seq_1hot
        self.seq_depth = seq_depth
        self.seq_length_crop = seq_length_crop

    def parse_proto(self, example_protos, raw=False):
        """Parse TFRecord protobuf."""

        # define features
        features = {
            TFR_INPUT: tf.io.FixedLenFeature([], tf.string),
            TFR_OUTPUT: tf.io.FixedLenFeature([], tf.string)
        }

        # parse example into features
        parsed_features = tf.io.parse_single_example(example_protos, features=features)

        # decode sequence
        sequence = tf.io.decode_raw(parsed_features[TFR_INPUT], tf.uint8)
#         if not raw:
#             if self.seq_1hot:
#                 sequence = tf.reshape(sequence, [self.seq_length])
#                 sequence = tf.one_hot(sequence, 1+self.seq_depth, dtype=tf.uint8)
#                 sequence = sequence[:,:-1]  # drop N
#             else:
#                 sequence = tf.reshape(sequence, [self.seq_length, self.seq_depth])
#             if self.seq_length_crop is not None:
#                 crop_len = (self.seq_length - self.seq_length_crop) // 2
#                 sequence = sequence[crop_len:-crop_len,:]
#             sequence = tf.cast(sequence, tf.uint8)

        # decode targets
        targets = tf.io.decode_raw(parsed_features[TFR_OUTPUT], tf.float16)
        if not raw:
            targets = tf.reshape(targets, [self.target_length, self.num_targets])
            targets = tf.cast(targets, tf.float32)

        return sequence, targets

In [4]:
TFR_INPUT = 'sequence'  # This needs to be the actual name used in your TFRecords for the sequence
TFR_OUTPUT = 'target'  # This needs to be the actual name used in your TFRecords for the targets

tfrs_path = '/clusterfs/nilah/richard/basenji2/data_pretrained_basenji2/human/tfrecords/'

In [5]:
tfrs_outpath = '/clusterfs/nilah/oberon/datasets/basenji2_human_training/dummy_tfrecords/'

In [6]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def feature_bytes(values):
    """Convert numpy arrays to bytes features."""
    values = values.flatten().tobytes()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))
def serialize_example(sequence, target):
    target = np.array(target).astype('float16')
    serialized_sequence = sequence.numpy().tobytes()
    
    features_dict = {
        TFR_INPUT: tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_sequence])),
        TFR_OUTPUT: feature_bytes(target)
    }
    example = tf.train.Example(features=tf.train.Features(feature=features_dict))
    return example

In [8]:
for tfr_filename in tqdm(tfr_filenames):
    parser = Parser()
    dataset = tf.data.TFRecordDataset(tfrs_path+tfr_filename,compression_type='ZLIB')
    parsed_dataset = dataset.map(parser.parse_proto)
    options = tf.io.TFRecordOptions(compression_type="ZLIB")
    with tf.io.TFRecordWriter(tfrs_outpath+tfr_filename,options=options) as writer:
        for sequence,targets in tqdm(parsed_dataset,total=256):
    #         print(targets.shape,type(targets))
            new_targets = tf.zeros((896,5313))
    #         print(new_targets.shape,type(new_targets))
            example = serialize_example(sequence,new_targets)
    #         print(example.SerializeToString()[0:100])
            writer.write(example.SerializeToString())

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

...

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]